# ArcGIS Feature Layer Field Modification

### Instructions 2 Run Notebook

1. Click **Kernal**
2. Click **Restart & Run All**

#### Example to Write Domain Coded List

```json
{'type': 'codedValue',
 'name': 'OWNER_Commnuity',
 'codedValues': [{'name': 'Name1', 'code': '1'}, 
                 {'name': 'Name 2', 'code':'Name2']}
                ]
}
```

In [1]:
%%HTML
<script>
  code_show = true; 
  function code_toggle() {
    if (code_show){
      $('div.input').hide();
    } else {
      $('div.input').show();
    }
    code_show = !code_show;  
  }
  $(document).ready(code_toggle);
</script>
<script type="module" src="https://js.arcgis.com/calcite-components/1.0.7/calcite.esm.js"></script>
<link rel="stylesheet" type="text/css" href="https://js.arcgis.com/calcite-components/1.0.7/calcite.css"/>
<button id="do_run_all" onclick="javascript:code_toggle()">Click here to toggle on/off the raw code.</button>

In [10]:
from arcgis.gis import GIS
import logging, warnings
import arcgis

# Set logger to error only
logger = logging.getLogger()
logger.setLevel(logging.ERROR)

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')


####################################################
## Help functions for processing Schema modification
####################################################

def add_message(v):
    display(widgets.HTML(
        value=f"<div>{str(v)}</div>",
        placeholder='',
        description='',
    ))
    
def add_error(v):
    display(widgets.HTML(
        value=f"<div style='color:red;'>{str(v)}</div>",
        placeholder='',
        description='',
    ))

    
def initialize():
    from time import sleep
    add_message('Initializing....')
    sleep(5)
    clear_output(wait=True)
    
def sqlType(key):
    """
        User passes an esri field type, and the function returns
        a sql type. 
    """
    type = {'esriFieldTypeOID': 'sqlTypeOther',
            'esriFieldTypeGlobalID': 'sqlTypeOther',
            'esriFieldTypeString': 'sqlTypeNVarchar', 
            'esriFieldTypeInteger': 'sqlTypeOther',
            'esriFieldTypeSmallInteger': 'sqlTypeOther',
            'esriFieldTypeSingle': 'sqlTypeOther', 
            'esriFieldTypeDouble': 'sqlTypeFloat', 
            'esriFieldTypeDate': 'sqlTypeTimestamp2'}
    return type[key]
####################################################
## Key functions for processing Schema modification
####################################################

def getFields(b):
    from time import sleep
    global attributes
    try:
        update_display()
        add_message('Making connection & Getting service...')
        # Set GIS AGOL connection
        gis = GIS('Home')
    
        # Get the Feature Layer & Fields
        fs = arcgis.features.FeatureLayer(input_fs.value, gis)
        add_message(f'Loading fields from {fs.properties.name.title()} Layer for Dropdown....')
        
        # Display progress bar to provide feedback to user on getting fields
        display(progress)
        comboList = [] # Create empty list to store fields & loops through FL to get them
        progress.max = len([i for i in fs.properties.fields]) # Count total features
        for i,v in enumerate(fs.properties.fields, 1):
            progress.value += 1
            comboList.append(v['name'])
            sleep(.5)
        progress.bar_style = 'success'
        progress.description = 'Success:'
        
        input_fs_fieldC.options = comboList
        input_fs_fieldC.disabled = False
    except Exception as error:
        add_error(error)
        progress.description = 'Error:'
        progress.bar_style = 'danger'

def click_populate_button(b):
    from time import sleep
    try:
        update_display()
        add_message(f'Populating attributes from {input_fs_fieldC.value} field...')
        display(progress)
        progress.max = 10
        progress.value = 2
        
         # Set GIS AGOL connection
        gis = GIS('Home')
    
        # Get the Feature Layer & Fields
        fs = arcgis.features.FeatureLayer(input_fs.value, gis)
        attributes = dict
        # Create Dictionary to store attributes key/value
        for field in fs.manager.properties['fields']:
            if input_fs_fieldC.value.lower() in field['name'].lower():
                attributes = field
        # Set progress bar to show 50% then Success at the end
        progress.value = 5      
        
        input_fs_name.value = attributes['name']
        input_fs_name.disabled = False
        
        input_fs_type.value = attributes['type']
        input_fs_type.disabled = False
        
        input_fs_alias.value= attributes['alias']
        input_fs_alias.disabled = False
        
        if attributes['sqlType'] in input_fs_sqlType.options:
            input_fs_sqlType.value =  attributes['sqlType']
        else:
            input_fs_sqlType.options = [i for i in input_fs_sqlType.options] + [attributes['sqlType']]
            input_fs_sqlType.value =  attributes['sqlType']
        input_fs_sqlType.disabled = False
            
        
        input_fs_length.value = str(attributes['length']) if 'length' in attributes.keys() else 0
        if 'length' in attributes.keys():
            input_fs_length.disabled = False
            
        input_fs_nullable.value = attributes['nullable']
        input_fs_nullable.disabled = False
        
        input_fs_editable.value = attributes['editable']
        input_fs_editable.disabled = False
        progress.value = 7
        input_fs_visible.value = True if 'visible' not in attributes.keys() else attributes['visible']
        input_fs_visible.default = False
        
        input_fs_domain.value = str(attributes['domain']) if bool(attributes['domain']) else 'None'
        input_fs_domain.disabled = False
        
        input_f_dfV.value = 'None' if not bool(attributes['defaultValue']) else str(attributes['defaultValue'])
        input_f_dfV.disabled = False
        
        sleep(1)
        progress.value = 10
        progress.bar_style ='success'
        progress.description = 'Success:'
        
    except Exception as error:
        add_error(error)
        progress.description = 'Error:'
        progress.bar_style = 'danger'

def click_submit(b):
    from time import sleep
    try:
        update_display()
        add_message(f'Submitting attributes modification for {input_fs_fieldC.value} field...')
        display(progress)
        progress.max = 10
        progress.value = 2
        
         # Set GIS AGOL connection
        gis = GIS('Home')
        # Get the Feature Layer & Fields
        fs = arcgis.features.FeatureLayer(input_fs.value, gis)
        
        # Set progress bar to show 50% then Success at the end
        progress.value = 5
        
        field_ =  [f for f in fs.manager.properties['fields'] if input_fs_name.value.lower() in f['name'].lower()][0].keys()
        if 'length' in field_:
            progress.value = 8
            field = {
                    "fields":[
                {
                  "name": input_fs_name.value,
                  "type": input_fs_type.value,
                  "alias": input_fs_alias.value,
                  "sqlType": input_fs_sqlType.value,
                  "length": input_fs_length.value,
                  "nullable": input_fs_editable.value,
                  "editable": input_fs_nullable.value,
                  "visible": input_fs_visible.value,
                  "domain": None  if input_fs_domain.value == 'None' else eval(input_fs_domain.value),
                  "defaultValue": None if input_f_dfV.value == 'None' else input_f_dfV
                }]}
            fs.manager.update_definition(field)
        else:
            progress.value = 8
            field = {
                    "fields":[
                {
                  "name": input_fs_name.value,
                  "type": input_fs_type.value,
                  "alias": input_fs_alias.value,
                  "sqlType": input_fs_sqlType.value,
                  "nullable": input_fs_nullable.value,
                  "editable": input_fs_editable.value,
                  "visible": input_fs_visible.value,
                  "domain": None  if input_fs_domain.value == 'None' else eval(input_fs_domain.value),
                  "defaultValue": None if input_f_dfV.value == 'None' else input_f_dfV
                }]}
            fs.manager.update_definition(field)
       
        sleep(.5)
        progress.value = 10
        progress.bar_style ='success'
        progress.description = 'Success:'
    except Exception as error:
        add_error(error)
        progress.description = 'Error:'
        progress.bar_style = 'danger'
        
def click_clear_button(b):
    # "linking function with output"
    update_display() 

In [11]:
import ipywidgets as widgets
from IPython.display import display, clear_output

## The Thematic Break (Horizontal Rule) element
hr = widgets.HTML(
    value="<hr style='height:1px; border:none; color:#EAEAEA; background-color:#EAEAEA'>",
    placeholder='',
    description='',
)
header = widgets.HTML(description='',value='<h1>Modify Fields</h1>')
## Helper notice
info_notice = widgets.HTML(f'''
  <calcite-notice style="float: left;" active open kind="success" icon="lightbulb" scale="l" width="auto">
      <div style="font-size:1.5rem;" slot="message">Enter Rest Serivce End Point URL.</div>
  </calcite-notice>
''')
info_notice2 = widgets.HTML(f'''
  <calcite-notice style="float: left;" active open kind="success" icon="lightbulb" scale="l" width="auto">
      <div style="font-size:1.5rem;" slot="message">Click Get Serivce button to populate the Field dropdown.</div>
  </calcite-notice>
''')


def update_display():
    clear_output(wait=True)
    progress.bar_style= ''
    progress.description = 'Progress:'
    display(widgets.VBox([header, info_notice,info_notice2, hr, rest_service_url, hr,accordion,hr,buttons,outW]))

initialize()


progress = widgets.IntProgress(
    value=0,
    min=0,
    max=10,
    step=1,
    description='Progress:',
    bar_style='', # 'success', 'info', 'warning', 'danger' or ''
    orientation='horizontal'
)
####################################################
# Input Feature Layer UI elements URL & Feilds
####################################################
input_fs_label = widgets.Label('Rest Service Endpoint URL')
input_fs =widgets.Text(
    value=r'https://services.arcgis.com/6I1ysurtNWNxkuwd/arcgis/rest/services/RoadTypeCopy/FeatureServer/0',
    placeholder='Provide the Rest URL end point',
    #description='Rest Service Endpoint:',
    disabled=False,
    layout = widgets.Layout(width='50')
)

input_fs_fieldC_label = widgets.Label('Field')
input_fs_fieldC = widgets.Dropdown(
    value='Select Field',
    options=['Select Field'], 
    disabled=True 
)

####################################################
# Feature Attributes UI elements for Modification
####################################################

input_fs_name = widgets.Text(
    value=None,
    placeholder='ArcGIS Layer Name',
    description='Name:',
    disabled=True,
    layout = widgets.Layout(width='50')
)

input_fs_type = widgets.Dropdown(
    options=['esriFieldTypeOID', 'esriFieldTypeString', 'esriFieldTypeInteger',
             'esriFieldTypeSmallInteger','esriFieldTypeSingle','esriFieldTypeDouble',
             'esriFieldTypeDate'],
    description='Field Type:',
    disabled=True,
    value = 'esriFieldTypeString'
)


# input_fs_type = widgets.Text(
#     value=None,
#     placeholder='ArcGIS Solution Value Type',
#     description='Type:',
#     disabled=True,
#     layout = widgets.Layout(width='50')
# )

input_fs_alias = widgets.Text(
    value=None,
    placeholder='ArcGIS Layer Alias',
    description='Name:',
    disabled=True,
    layout = widgets.Layout(width='50')
)

input_fs_sqlType = widgets.Dropdown(
    options=['sqlTypeOther', 'sqlTypeNVarchar', 'sqlTypeTimestamp2','sqlTypeFloat'
             'sqlTypeInteger','sqlTypeGUID'],
    description='Sql Type:',
    disabled=True,
    value = 'sqlTypeOther'
)

# input_fs_sqlType = widgets.Text(
#     value='sqlTypeOther',
#     placeholder='ArcGIS SQL Type',
#     description='Type:',
#     disabled=True,
#     layout = widgets.Layout(width='50')
# )

input_fs_length = widgets.BoundedIntText(
    value=None,
    min=0,
    max=255,
    step=1,
    placeholder='ArcGIS Length',
    description='Length:',
    disabled=True,
    layout = widgets.Layout(width='50')
)


input_fs_nullable = widgets.Checkbox(
    value = True,
    description= 'Allow',
    disabled = True
)
input_fs_editable = widgets.Checkbox(
    value = True,
    description= 'Editable',
    disabled = True
)
input_fs_visible = widgets.Checkbox(
    value = True,
    description= 'Visible',
    disabled = True
)


input_fs_domain = widgets.Textarea(
    value="[{'code1': 'value1'}, {'code2':value2'}]",
    placeholder='ArcGIS Domain',
    description='Domain:',
    disabled=True,
    layout = widgets.Layout(width='50')
)
input_f_dfV = widgets.Text(
    value=None,
    placeholder='ArcGIS Default Value',
    description='SQL Type:',
    disabled=True,
    layout = widgets.Layout(width='50')
)

# Outwidget
outW = widgets.Output()

####################################################
# Buton UI elements 
####################################################
gButton = widgets.Button(
    value = False,
    description='Get Service',
    disabled=False,
    tooltip='Button Gets Service End Point',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
sButton = widgets.Button(
    value = False,
    description='Submit Modification',
    disabled=False,
    tooltip='Submit Field Modification',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
cButton = widgets.Button(
    description='Clear',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Clear output',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)
pfButton = widgets.Button(
    value = False,
    description='Populate Attribute',
    disabled=False,
    tooltip='Button Gets Service End Point',
    #icon='check' # (FontAwesome names without the `fa-` prefix)
)


rest_service_url = widgets.HBox([widgets.VBox([input_fs_label, input_fs,]) , widgets.VBox([input_fs_fieldC_label,input_fs_fieldC])])

accordion = widgets.Accordion(children=[input_fs_name, input_fs_type, input_fs_alias, input_fs_sqlType, input_fs_length, input_fs_nullable, input_fs_editable, input_fs_visible, input_fs_domain,input_f_dfV])
for index, title in enumerate(['Layer Name','Field Type','Alias','SQL Type', 'Length', 'Is Nullable', 'Editable', 'Field Visible', 'Domain', 'Default Value']):
    accordion.set_title(index, title)

buttons = widgets.HBox([gButton,pfButton,sButton,cButton])


gButton.on_click(getFields)
pfButton.on_click(click_populate_button)
sButton.on_click(click_submit)
cButton.on_click(click_clear_button)

update_display()

HTML(value='<div>Populating attributes from OBJECTID field...</div>', placeholder='')

IntProgress(value=19, description='Progress:', max=19)